In [38]:
import pandas as pd
import numpy as np

In [125]:
X = pd.read_csv('../Heatmaps/RCS_compiled.csv')
print(len(X))
X.head()

27927


,Unnamed: 0,id,organization,last_name,first_name,email,phone,gender,ethnicity,grad_year,aptitude_fit,interest_fit,fit_rank
0,0,409512,Rocky Fork Middle School,Cardoso,Sofia,scardosoperez8225@student.rcschools.net-moved-...,615-900-7342,f,other,2023.0,Agriculture & Natural Resources,Human Services,3
1,1,409512,Rocky Fork Middle School,Cardoso,Sofia,scardosoperez8225@student.rcschools.net-moved-...,615-900-7342,f,other,2023.0,Finance,Arts & Media,2
2,2,409512,Rocky Fork Middle School,Cardoso,Sofia,scardosoperez8225@student.rcschools.net-moved-...,615-900-7342,f,other,2023.0,Engineering,Teaching,1
3,3,409514,Rocky Fork Middle School,Acevedo,Ubaldo,ubarajas5459@student.rcschools.net-moved-to-hs,615-904-6780,m,hispanic,2023.0,Agriculture & Natural Resources,Law & Public Safety,3
4,4,409514,Rocky Fork Middle School,Acevedo,Ubaldo,ubarajas5459@student.rcschools.net-moved-to-hs,615-904-6780,m,hispanic,2023.0,Architecture & Construction,Arts & Media,2


In [104]:
len(X[X.fit_rank == 1])

9309

In [101]:
test = X[X.aptitude_fit == 'Hospitality & Tourism']
# test = test[test.fit_rank == 1]
test

,Unnamed: 0,id,organization,last_name,first_name,email,phone,gender,ethnicity,grad_year,aptitude_fit,interest_fit,fit_rank
78,78,410241,Rocky Fork Middle School,Stranko,Mary,mstranko8945@student.rcschools.net-moved-to-hs,629-867-4737,f,other,2023.0,Hospitality & Tourism,Human Services,3
90,90,410301,Rocky Fork Middle School,Brown,Hailey,hbrown4051@student.rcschools.net-moved-to-hs,161-526-0408,f,white,2023.0,Hospitality & Tourism,Law & Public Safety,3
111,111,410409,Rocky Fork Middle School,smith,jiane,jsmith8477@student.rcschools.net-moved-to-hs,832-301-2877,f,black,2023.0,Hospitality & Tourism,Arts & Media,3
126,126,411996,Rocky Fork Middle School,GRIMES,MIGUEL,mgrimes6285@student.rcschools.net-moved-to-hs,615-683-8845,m,hispanic,2023.0,Hospitality & Tourism,Architecture & Construction,3
300,300,413868,Rocky Fork Middle School,zewde,nahom,nzewde666@student.rcschools.net-moved-to-hs,615-915-7708,m,black,2023.0,Hospitality & Tourism,Agriculture & Natural Resources,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27723,27723,1490426,Oakland Middle School,BARRETT,GRACIE,gbarrett9685@student.rcschools.net,615-557-0201,f,white,2026.0,Hospitality & Tourism,Human Services,3
27759,27759,1490465,Oakland Middle School,DELFIN VALERIO,YAJAIRA,ydelfinvalerio000@student.rcschools.net,615-546-2012,f,hispanic,2026.0,Hospitality & Tourism,Law & Public Safety,3
27798,27798,1490505,Oakland Middle School,NANCE,ELLIE,enance6859@student.rcschools.net,615-289-8455,f,white,2026.0,Hospitality & Tourism,Arts & Media,3
27879,27879,1490719,Oakland Middle School,HENDERSON,NIKILYA,nhenderson001@student.rcschools.net,615-904-6760,f,other,2026.0,Hospitality & Tourism,Architecture & Construction,3


In [69]:
def get_breakdown_by_rank(df, rank=1):
    # compiling frequency of interest/aptitude pairs
    
    fits = list(set(df.aptitude_fit).union(df.interest_fit))
    df = df[df.fit_rank == rank]
    
    breakdown_df = None
    for fit in fits:
        Y = df.loc[X.aptitude_fit == fit].interest_fit.value_counts()
        Y = pd.DataFrame(Y)
        Y.rename(columns={'interest_fit':'Apt: ' + fit},inplace=True)

        if breakdown_df is None: 
            breakdown_df = Y
        else:
            breakdown_df = pd.concat([breakdown_df,Y], axis=1)

    breakdown_df.replace(np.nan,0,inplace=True)
    
    return breakdown_df

In [103]:
r1_df = get_breakdown_by_rank(X)
util_dict = {}
for col in list(r1_df.columns):
    total = int(r1_df[col].sum())
    try:
        cat = r1_df[col].loc[col.split(': ')[1]]
        print(col, '- Total: ', total, '; %Interested: ', "{:.2%}".format(float(cat)/total))
    except KeyError:
        print(f'~~~At fit_rank 1, none were labeled with {col}')

Apt: Business - Total:  761 ; %Interested:  7.49%
Apt: Agriculture & Natural Resources - Total:  64 ; %Interested:  0.00%
~~~At fit_rank 1, none were labeled with Apt: Hospitality & Tourism
Apt: Engineering - Total:  629 ; %Interested:  18.28%
Apt: Teaching - Total:  1307 ; %Interested:  45.37%
Apt: Finance - Total:  145 ; %Interested:  0.69%
Apt: Distribution & Logistics - Total:  467 ; %Interested:  8.14%
Apt: Arts & Media - Total:  25 ; %Interested:  4.00%
Apt: Sales & Marketing - Total:  1027 ; %Interested:  21.52%
Apt: Architecture & Construction - Total:  6 ; %Interested:  0.00%
Apt: Computers & Technology - Total:  296 ; %Interested:  0.68%
~~~At fit_rank 1, none were labeled with Apt: Law & Public Safety
Apt: Health Science - Total:  3647 ; %Interested:  7.65%
Apt: Advanced Manufacturing - Total:  934 ; %Interested:  4.50%
~~~At fit_rank 1, none were labeled with Apt: Government & Public Admin
~~~At fit_rank 1, none were labeled with Apt: Human Services


In [113]:
list(r1_df.columns)


['Apt: Business',
 'Apt: Agriculture & Natural Resources',
 'Apt: Hospitality & Tourism',
 'Apt: Engineering',
 'Apt: Teaching',
 'Apt: Finance',
 'Apt: Distribution & Logistics',
 'Apt: Arts & Media',
 'Apt: Sales & Marketing',
 'Apt: Architecture & Construction',
 'Apt: Computers & Technology',
 'Apt: Law & Public Safety',
 'Apt: Health Science',
 'Apt: Advanced Manufacturing',
 'Apt: Government & Public Admin',
 'Apt: Human Services']

In [105]:
# checking no students were dropped 
sum = 0
for col in list(r1_df.columns):
    sum += r1_df[col].sum()
sum

9309.0

In [110]:
# how many students across RCS had interest/aptitude align with fit_rank 1? 
perfect_fit = 0
for col in list(r1_df.columns):
    try:
        perfect_fit += int(r1_df[col].loc[col.split(': ')[1]])   
    except KeyError:
        pass

print(perfect_fit, 9309-perfect_fit, "{:.2%}".format(perfect_fit/7960))

# This does not mean that those 1349 could get into their appropriate POS placements

1349 7960 16.95%


In [117]:
list(set(X.organization))

['Oakland Middle School',
 'Siegel Middle School',
 'Whitworth-Buchanan Middle School',
 'Christiana Middle School',
 'LaVergne Middle School',
 'Rocky Fork Middle School',
 'Thurman Francis Arts Academy',
 'Rock Springs Middle School',
 'Rockvale Middle School',
 'Smyrna Middle School',
 'Blackman Middle School',
 'Stewarts Creek Middle School']

In [126]:
Z = X[X.organization == 'LaVergne Middle School']
# compiling frequency of interest/aptitude pairs
fits = list(set(X.aptitude_fit).union(X.interest_fit))
breakdown_df = None
for fit in fits:
    Y = Z.loc[Z.aptitude_fit == fit].interest_fit.value_counts()
    Y = pd.DataFrame(Y)
    Y.rename(columns={'interest_fit':'Apt: ' + fit},inplace=True)

    if breakdown_df is None: 
        breakdown_df = Y
    else:
        breakdown_df = pd.concat([breakdown_df,Y], axis=1)

breakdown_df.replace(np.nan,0,inplace=True)

breakdown_df

,Apt: Business,Apt: Agriculture & Natural Resources,Apt: Hospitality & Tourism,Apt: Engineering,Apt: Teaching,Apt: Finance,Apt: Distribution & Logistics,Apt: Arts & Media,Apt: Sales & Marketing,Apt: Architecture & Construction,Apt: Computers & Technology,Apt: Law & Public Safety,Apt: Health Science,Apt: Advanced Manufacturing,Apt: Government & Public Admin,Apt: Human Services
Teaching,23.0,2,0.0,11.0,62.0,3,14.0,2,56.0,0.0,12.0,0.0,147.0,26,0.0,0.0
Sales & Marketing,22.0,5,0.0,6.0,35.0,16,8.0,4,41.0,2.0,10.0,0.0,105.0,29,1.0,0.0
Health Science,14.0,15,0.0,13.0,7.0,30,9.0,5,10.0,11.0,21.0,0.0,38.0,22,0.0,3.0
Arts & Media,13.0,67,9.0,7.0,2.0,58,1.0,16,6.0,19.0,39.0,19.0,31.0,47,14.0,19.0
Business,10.0,15,2.0,5.0,5.0,33,5.0,3,18.0,5.0,22.0,1.0,33.0,30,8.0,3.0
Engineering,10.0,5,0.0,9.0,15.0,20,4.0,3,15.0,7.0,18.0,0.0,48.0,33,1.0,1.0
Computers & Technology,6.0,16,0.0,4.0,3.0,12,6.0,6,2.0,7.0,8.0,3.0,11.0,21,2.0,3.0
Human Services,5.0,102,7.0,6.0,0.0,41,1.0,25,3.0,16.0,30.0,35.0,5.0,29,22.0,15.0
Advanced Manufacturing,4.0,2,0.0,1.0,4.0,4,3.0,2,3.0,2.0,2.0,0.0,20.0,13,0.0,1.0
Finance,3.0,20,3.0,1.0,2.0,12,1.0,2,1.0,3.0,8.0,4.0,8.0,15,1.0,4.0


In [127]:
int_total, apt_total = {}, {}
print('Interest | Aptitude | Category')
for fit in fits:
    int_total[fit] = breakdown_df.loc[fit].sum()
    apt_total['Apt: ' + fit] = breakdown_df['Apt: ' + fit].sum()
    
    print(int_total[fit], apt_total['Apt: ' + fit], fit)

Interest | Aptitude | Category
198.0 118.0 Business
171.0 483 Agriculture & Natural Resources
71.0 58.0 Hospitality & Tourism
189.0 80.0 Engineering
358.0 152.0 Teaching
88.0 326 Finance
143.0 57.0 Distribution & Logistics
367.0 108 Arts & Media
284.0 177.0 Sales & Marketing
85.0 117.0 Architecture & Construction
110.0 239.0 Computers & Technology
388.0 120.0 Law & Public Safety
198.0 504.0 Health Science
61.0 329 Advanced Manufacturing
13.0 109.0 Government & Public Admin
342.0 89.0 Human Services
